In [1]:
import pandas as pd
import psycopg2
import os, sys

vector_path = "SearchApp/QiitaSearch/DB/Qiita_vector_store.csv"


In [2]:
connection = psycopg2.connect("host=localhost dbname=newqiita user=postgres password=user1021")

In [3]:
cur = connection.cursor()

In [4]:
df = pd.read_csv(vector_path)
print(df.columns[:10])
df.shape

Index(['article_id', 'sub_text_id', 'v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6',
       'v7'],
      dtype='object')


(3123, 1538)

In [5]:
vector_size = len(df.loc[0,'v0':])
vector_df = df.loc[:, 'v0':].values.tolist()
article_id_df = df.loc[:,'article_id'].to_list()
sub_text_id_df = df.loc[:, 'sub_text_id'].to_list()
print(vector_size)

1536


In [33]:
#ベクトルデータ格納用のtableの作成

cur.execute(f"CREATE TABLE qiita_vector_store(article_id TEXT , sub_text_id TEXT , embedding vector({vector_size}), PRIMARY KEY (article_id, sub_text_id));")
connection.commit()

In [32]:
cur.execute(f"DROP TABLE IF EXISTS qiita_vector_store;")
connection.commit()

In [67]:
connection.rollback()

In [6]:
vector_data = ", ".join([f"('{a}', '{s}', '{v}')" for a,s,v in zip(article_id_df[:2], sub_text_id_df[:2], vector_df[:2])])

In [7]:
vector_data

"('a0', 's0', '[-0.003177247243002, -0.0030844227876514, -0.0331834033131599, -0.028494443744421, 0.0460409112274646, -0.0630993843078613, -0.0363235212862491, 0.0274972449988126, 0.0025526713579893, 0.0225112475454807, 0.0073092598468065, -0.0047711748629808, -0.0239540040493011, -0.0063810157589614, -0.0096643473953008, -0.019604517146945, 0.0201243329793214, -0.0177692454308271, -0.0430705286562442, 0.0036917596589773, 0.0165068339556455, -0.0222778599709272, 0.0205168481916189, -0.0076911658979952, 0.012719596736133, -0.0170054323971271, 0.0150428600609302, -0.0018816834781318, -0.0134091498330235, -0.0283034909516572, -0.077611818909645, -0.0142896557226777, -0.0123695163056254, -0.0226597655564546, -0.013101503252983, -0.0335653088986873, 0.0074789957143366, 0.0002902420528698, -0.00158729753457, 0.0007770729716867, 0.0322074219584465, -0.0357930958271026, 0.0358991809189319, 0.017928373068571, -0.0279003679752349, 0.059237889945507, -0.0471017621457576, -0.0366629920899868, 0.05

In [10]:
vector_data = ", ".join([f"('{a}', '{s}', '{v}')" for a,s,v in zip(article_id_df, sub_text_id_df, vector_df)])
cur.execute(f"INSERT INTO qiita_vector_store (article_id, sub_text_id, embedding) VALUES {vector_data};")

In [16]:
cur.execute("SELECT embedding FROM qiita_vector_store WHERE article_id = 'a1'")
cur.fetchall()


[('[0.027840588,0.01647615,-0.015335144,0.051162783,0.09684873,-0.036284044,-0.018176252,0.02834263,-0.042285744,0.021416714,0.059743162,0.04007219,-0.018769577,0.0038965412,0.051619187,-0.005807729,0.021827478,-0.074074216,-0.010913739,0.0412132,0.05810011,0.042719327,0.022067089,0.010987904,-0.03402485,-0.004946268,0.020070326,0.029894402,-0.0040676924,-0.03379665,0.0092307525,-0.028616473,-0.0030664583,-0.03162873,0.03311204,0.043928795,0.011432897,-0.008483392,0.06362259,0.040733974,0.05942368,-0.022740284,0.015711676,0.04169242,-0.04027757,-0.0048721028,-0.024759866,0.018735345,0.034230232,0.017377548,-0.057415508,0.0532166,-0.014958611,0.012208782,0.013943114,-0.0018484325,-0.012699416,0.022546312,-0.043883156,0.009892536,0.031491812,0.005476837,0.00885422,0.03712839,-0.023459118,0.0046496065,-0.019796483,-0.038429137,-0.043426752,0.011238926,0.003500041,-0.0082608955,-0.053809922,0.006498039,-0.0010518664,-0.028000329,0.0075135357,0.02813725,0.023128226,-0.0117409695,-0.01203763

In [14]:
connection.commit()

In [17]:
def sql_exe(sql, values=None):
    try:
        cur.execute(sql, values)
    except psycopg2.Error as e:
        print("SQLエラー", e)
        connection.rollback()
        return
    return 

In [19]:
#ベクトル検索

#option---L2:L2ノルム(L2距離)
#       |-L1:L1ノルム(L1距離)
#       |-Cos:コサイン類似度
#       |-In:内積

def vec_search(table, columns, query_vec, option='L2', top_n=1):
    if len(query_vec) != vector_size:
        print("ベクトルの次元数が違います")
        return
    if option == 'L2':
        option = '<->'
    elif option == 'Cos':
        option = '<=>'
    elif option == 'L1':
        option = '<+>'
    elif option == 'In':
        option = '<#>'
    else:
        print('指定のないパラメータです')
    if isinstance(columns, list):
        columns = ", ".join(columns)
    if option in ['<->', '<+>']:
        sql = f'SELECT {columns}, embedding {option} \'{query_vec}\'::vector AS distance FROM {table} ORDER BY distance LIMIT {top_n};'
    elif option == '<#>':
        sql = f'SELECT {columns}, (embedding {option} \'{query_vec}\'::vector)*-1 AS inner_product FROM {table} ORDER BY inner_product DESC LIMIT {top_n};'
    elif option == '<=>':
        sql = f'SELECT {columns}, (1-(embedding {option} \'{query_vec}\'::vector)) AS cosine_sim FROM {table} ORDER BY cosine_sim DESC LIMIT {top_n};'
    sql_exe(sql)
    res = cur.fetchall()
    return res

In [69]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

query = input("クエリを入力してください:")
response = client.embeddings.create(input=query,
                                    model="text-embedding-3-small")
query_embedding = response.data[0].embedding



In [70]:
results = vec_search('qiita_vector_store', ['article_id','sub_text_id'], query_embedding, option='Cos', top_n=10)

In [71]:
print(results)

[('a287', 's0', nan), ('a263', 's0', nan), ('a152', 's0', nan), ('a87', 's0', nan), ('a188', 's0', nan), ('a190', 's0', nan), ('a133', 's0', nan), ('a138', 's0', nan), ('a78', 's0', nan), ('a312', 's0', nan)]


In [72]:
result_articles = ", ".join([f"'{article_id[0]}'" for article_id in zip([result[0] for result in results])])
print(result_articles)
cur.execute(f"SELECT title FROM article_db WHERE article_id IN ({result_articles});")
res = cur.fetchall()
print(res)

'a287', 'a263', 'a152', 'a87', 'a188', 'a190', 'a133', 'a138', 'a78', 'a312'
[('What Is Machine Learning and How Does It Work?',), ('### How to Install Python 2 on Ubuntu 20.04',), ('selfを省略してても呼び出せる',), ('Free consultation with GPT',), ('Looking into the future of creativity and innovation in Generative AI World 2024',), ('Data Engineering Tools in 2025',), ('Claudeでcommand_executorを使う',), ('Azure Logic Apps のSQL Server アクションで、パラメータクエリを使用する',), ('Mental Health and Behavior: Understanding the Connection',), ('tkinopi/browser-use-create-tweet-from-latest-news',)]


In [80]:
cur.execute(f"SELECT title FROM article_db WHERE article_id = 'a78';")
cur.fetchall()

[('What Is Machine Learning and How Does It Work?',)]